# ETF相关知识

## 国内ETF一共有多少只？其中已经上市的ETF数量是多少？
* 上交所
    * ETF列表：http://www.sse.com.cn/assortment/fund/etf/list/
    * 指数对应的成分股：http://www.sse.com.cn/assortment/fund/list/etfinfo/redemptionlist/index.shtml?FUNDID=510630
* 深交所
    * ETF列表：http://www.szse.cn/market/fund/list/etfList/index.html
    * 指数与样本股：http://www.szse.cn/market/exponent/sample/index.html